In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
import calendar
from scipy import stats

In [2]:
from matplotlib import pyplot as plt
import seaborn as sn
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [3]:
#for exponentional model
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [4]:
#for ARMIA model
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [5]:
#for Linearregression model
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression

In [6]:
#for Randomforest model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [20]:
train = pd.read_csv('C:\\Users\\lenovo\\Documents\\Feynn Labs Internship\\Project 4\\train (2).csv')
test=pd.read_csv('C:\\Users\\lenovo\\Documents\\Feynn Labs Internship\\Project 4\\test (2).csv')
sale=pd.read_csv('C:\\Users\\lenovo\\Documents\\Feynn Labs Internship\\Project 4\\sample_submission.csv')

In [26]:
train.head()

,id,date,store,item,Sales
0,0,1/1/2018,1,1,13
1,1,1/2/2018,1,1,14
2,2,1/3/2018,1,1,15
3,3,1/4/2018,1,1,16
4,4,1/5/2018,1,1,17


In [22]:
test.head()

,id,date,store,item
0,0,2018-01-01,1,1
1,1,2018-01-02,1,1
2,2,2018-01-03,1,1
3,3,2018-01-04,1,1
4,4,2018-01-05,1,1


# Cleaning the data

In [27]:
train.isnull().sum()

id       0
date     0
store    0
item     0
Sales    0
dtype: int64

In [28]:
test.isnull().sum()

id       0
date     0
store    0
item     0
dtype: int64

Data preprocessing

In [29]:
# Combine train and test data into a single DataFrame
whole = train
print(train)

          id       date  store  item  Sales
0          0   1/1/2018      1     1     13
1          1   1/2/2018      1     1     14
2          2   1/3/2018      1     1     15
3          3   1/4/2018      1     1     16
4          4   1/5/2018      1     1     17
...      ...        ...    ...   ...    ...
44995  44995  3/27/2018     10    50     92
44996  44996  3/28/2018     10    50     93
44997  44997  3/29/2018     10    50     94
44998  44998  3/30/2018     10    50     95
44999  44999  3/31/2018     10    50     96

[45000 rows x 5 columns]


In [30]:
train['date'] = pd.to_datetime(train['date'], format='%Y-%m-%d')  # convert date column to datetime object
train['year'] = train['date'].dt.year  

test['date'] = pd.to_datetime(test['date'], format='%Y-%m-%d')  # convert date column to datetime object
test['year'] = test['date'].dt.year
print(test['year'])

ValueError: time data 1/1/2018 doesn't match format specified

In [15]:
train = train[(train['store'] == 1) & (train['item'] == 1)]
train['date'] = pd.to_datetime(train['date'], format='%Y-%m-%d')  
train['year'] = train['date'].dt.year  
train['month'] = train['date'].dt.month  
train['day'] = train['date'].dt.day  
train['weekday'] = train['date'].dt.weekday  
train['weekday'] = np.where(train.weekday == 0, 7, train.weekday)  

KeyError: 'store'

In [31]:
train.head()

,id,date,store,item,Sales
0,0,1/1/2018,1,1,13
1,1,1/2/2018,1,1,14
2,2,1/3/2018,1,1,15
3,3,1/4/2018,1,1,16
4,4,1/5/2018,1,1,17


In [32]:
test = test[(test['store'] == 1) & (test['item'] == 1)]
test['date'] = pd.to_datetime(test['date'], format='%Y-%m-%d')  # convert date column to datetime object
test['year'] = test['date'].dt.year 
test['month'] = test['date'].dt.month  
test['day'] = test['date'].dt.day
test['weekday'] = test['date'].dt.weekday  
test['weekday'] = np.where(test.weekday == 0, 7, test.weekday)

In [33]:
test.head()

,id,date,store,item,year,month,day,weekday
0,0,2018-01-01,1,1,2018,1,1,7
1,1,2018-01-02,1,1,2018,1,2,1
2,2,2018-01-03,1,1,2018,1,3,2
3,3,2018-01-04,1,1,2018,1,4,3
4,4,2018-01-05,1,1,2018,1,5,4


In [34]:
temp = train.set_index('date')
train= temp.loc[:'2017-09-30'].reset_index(drop=False)
test = temp.loc['2017-10-01':].reset_index(drop=False)

KeyError: '2017-09-30'

In [35]:
train.head()

,id,date,store,item,Sales
0,0,1/1/2018,1,1,13
1,1,1/2/2018,1,1,14
2,2,1/3/2018,1,1,15
3,3,1/4/2018,1,1,16
4,4,1/5/2018,1,1,17


In [36]:
test.head()

,id,date,store,item,year,month,day,weekday
0,0,2018-01-01,1,1,2018,1,1,7
1,1,2018-01-02,1,1,2018,1,2,1
2,2,2018-01-03,1,1,2018,1,3,2
3,3,2018-01-04,1,1,2018,1,4,3
4,4,2018-01-05,1,1,2018,1,5,4


In [37]:
#Visulaization

In [38]:
import matplotlib.pyplot as plt
import seaborn as sns

# Grouping data by weekday and calculating the sum of sales for each weekday
weekday_agg = train.groupby('weekday')['sales'].sum().reset_index()
# Creating the bar graph for weekday sales distribution
plt.figure(figsize=(4, 4))
sns.barplot(x='weekday', y='sales', data=weekday_agg)
plt.xlabel('Weekday')
plt.ylabel('Sum of Sales')
plt.title('Weekly Sales Distribution')
plt.show()
# Creating the line plot for weekly sales trend
plt.figure(figsize=(4, 4))
sns.lineplot(x='weekday', y='sales', data=weekday_agg)
plt.xlabel('Weekday')
plt.ylabel('Sales')
plt.title('Weekly Sales Trend')
plt.show()

KeyError: 'weekday'

In [39]:
# Grouping data by month and calculating the sum of sales for each month
monthly_agg = train.groupby('month')['sales'].sum().reset_index()

# Creating the bar graph
plt.figure(figsize=(4, 4))
sns.barplot(x='month', y='sales', data=monthly_agg)
plt.xlabel('Month')
plt.ylabel('Sum of Sales')
plt.title('Monthly Sales')
plt.show()

# Creating the line plot
plt.figure(figsize=(4, 5))
sns.lineplot(x='month', y='sales', data=monthly_agg)
plt.xlabel('Month')
plt.ylabel('Sales')
plt.title('Monthly Sales Trend')
plt.show()

KeyError: 'month'

In [40]:
# Grouping data by year and calculating the sum of sales for each year
yearly_agg = train.groupby('year')['sales'].sum().reset_index()
# Creating a figure with two subplots
fig, axs = plt.subplots(nrows=2, figsize=(4, 4))
# Bar graph for yearly sales distribution
sns.barplot(x='year', y='sales', data=yearly_agg, ax=axs[0])
axs[0].set_title('Yearly Sales Distribution')
axs[0].set_xlabel('Year')
axs[0].set_ylabel('Sales')
# Line plot for sales trend over the years
sns.lineplot(x='year', y='sales', data=yearly_agg, ax=axs[1])
axs[1].set_title('Sales Trend Over the Years')
axs[1].set_xlabel('Year')
axs[1].set_ylabel('Sales')

plt.tight_layout()
plt.show()

KeyError: 'year'

In [41]:
# Extracting the day from the date
train['day'] = train['date'].dt.day

# Grouping data by date and calculating the sum of sales for each date
daily_agg = train.groupby('date')['sales'].sum().reset_index()

# Creating the bar graph for daily sales distribution
plt.figure(figsize=(9, 7))
sns.boxplot(x='day', y='sales', data=train)
plt.title('Daily Sales Distribution')
plt.xlabel('Day')
plt.ylabel('Sales')
plt.show()

# Creating the line plot for daily sales trend
plt.figure(figsize=(9, 7))
sns.lineplot(x='date', y='sales', data=daily_agg)
plt.title('Daily Sales Trend')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.show()

AttributeError: Can only use .dt accessor with datetimelike values

In [42]:
# Extracting the day from the date
train['day'] = train['date'].dt.day

# Grouping data by day and calculating the sum of sales for each day
daily_agg = train.groupby('day')['sales'].sum().reset_index()

# Creating the bar graph for daily sales distribution
plt.figure(figsize=(8, 7))
sns.barplot(x='day', y='sales', data=daily_agg)
plt.title('Daily Sales Distribution')
plt.xlabel('Day')
plt.ylabel('Sales')
plt.show()

# Creating the line plot for daily sales trend
plt.figure(figsize=(8, 7))
sns.lineplot(x='day', y='sales', data=daily_agg)
plt.title('Daily Sales Trend')
plt.xlabel('Day')
plt.ylabel('Sales')
plt.show()

AttributeError: Can only use .dt accessor with datetimelike values

In [43]:
# subtract 1 year from test data
dates = (test['date'] - np.timedelta64(1, 'Y') +  np.timedelta64(1, 'D')).values.astype('datetime64[D]') 
seasonal_naive_sales = train[train['date'].astype('datetime64[D]').isin(dates)]['sales'] 
# make a copy of the test_df and make naive predictions for the last 3 months of 2017
sn_pred_df = test.copy().drop('sales', axis=1)
sn_pred_df['seasonal_naive_sales'] = pd.DataFrame(seasonal_naive_sales).set_index(test.index)
sn_pred_df.head()
forecast_dates = pd.date_range(start='2018-01-01', end='2020-01-01', freq='D')
forecast_df = pd.DataFrame(index=forecast_dates, columns=['seasonal_naive_sales'])
forecast_df['seasonal_naive_sales'] = seasonal_naive_sales.mean()

KeyError: 'sales'

In [44]:
errors_df = pd.merge(test, sn_pred_df, on='date')
errors_df = errors_df[['date', 'sales', 'seasonal_naive_sales']]
errors_df = pd.merge(test, sn_pred_df, on='date')
errors_df = errors_df[['date', 'sales', 'seasonal_naive_sales']]
errors_df['errors'] = test['sales'] - sn_pred_df['seasonal_naive_sales']
errors_df.insert(0, 'model', 'Seasonal Naive') 

def mae(err):
    return np.mean(np.abs(err))

def rmse(err):
    return np.sqrt(np.mean(err ** 2))

def mape(err, sales=errors_df['sales']):
    return np.sum(np.abs(err))/np.sum(sales) * 100

result_df = errors_df.groupby('model').agg(total_sales=('sales', 'sum'),
                                           total_sn_pred_sales=('seasonal_naive_sales', 'sum'),
                                           overall_error=('errors', 'sum'),
                                           MAE=('errors', mae), 
                                           RMSE=('errors', rmse), 
                                           MAPE=('errors', mape))
plt.figure(figsize=(11,5))
plt.plot(errors_df['date'], np.abs(errors_df['errors']), label='errors')
plt.plot(errors_df['date'], errors_df['sales'], label='actual sales')
plt.plot(errors_df['date'], errors_df['seasonal_naive_sales'], label='forecast')
plt.legend(loc='best')
plt.xlabel('date')
plt.ylabel('sales')
plt.title('Seasonal Naive forecasts with actual sales and errors')
plt.show()

result_df

NameError: name 'sn_pred_df' is not defined

In [45]:
ts_decomp_df = train.set_index('date') # set date as index
ts_decomp_df['sales'] = ts_decomp_df['sales'].astype(float)
ts_decomp_df.head()

KeyError: 'sales'

In [46]:
# Perform seasonal decomposition
result = seasonal_decompose(ts_decomp_df['sales'], model='additive')
fig = plt.figure()
fig = result.plot()
fig.set_size_inches(11, 11)
plt.show()

KeyError: 'sales'

# 2.Holt Winter Triple Exponential Smoothing Model

In [47]:
import warnings
warnings.filterwarnings('ignore')
hw_train_df = train[['date', 'sales']].set_index('date')
hw_test_df = test[['date', 'sales']].set_index('date')
# Model 1: Triple Exponential Smoothing without damping
hw_model_1 = ExponentialSmoothing(hw_train_df, trend='add', seasonal='add', seasonal_periods=12, initialization_method="estimated")
hw_fit_1 = hw_model_1.fit()
pred_fit_1 = pd.Series(hw_fit_1.predict(start=hw_test_df.index[0], end=hw_test_df.index[-1]), 
                       name='pred_sales').reset_index()

# Model 2: Triple Exponential Smoothing with damping
hw_model_2 = ExponentialSmoothing(hw_train_df, trend='add', seasonal='add', seasonal_periods=12, damped=True, use_boxcox=False)
hw_fit_2 = hw_model_2.fit(remove_bias=False)
pred_fit_2 = pd.Series(hw_fit_2.predict(start=hw_test_df.index[0], end=hw_test_df.index[-1]), 
                       name='pred_sales').reset_index()
start_date_2018 = '2018-01-01'
end_date_2019 = '2020-01-01'
forecast_2019_1 = hw_fit_1.predict(start=start_date_2018, end=end_date_2019)
forecast_2019_2 = hw_fit_2.predict(start=start_date_2018, end=end_date_2019)
print('Forecasts made, ready for evaluation')

KeyError: "['sales'] not in index"

In [48]:
# Merge predictions and actual sales into one df
errors_df_hwd = pd.merge(test, pred_fit_2, left_on='date', right_on='index')
errors_df_hwd = errors_df_hwd[['date', 'sales','pred_sales']]
errors_df_hwd['errors'] = errors_df_hwd.sales - errors_df_hwd.pred_sales
errors_df_hwd.insert(0, 'model', 'Holt-Winters-Damped') 

# Evaluate the predictions for Holt-Winters without damping trend component
plt.figure(figsize=(11,5))
plt.plot(train['date'], train['sales'], label='Train')
plt.plot(test['date'], test['sales'], label='Test')
plt.plot(errors_df_hwd['date'], errors_df_hwd['pred_sales'], label='Forecast - HW damping')
plt.plot(forecast_2019_2.index, forecast_2019_2, label='Forecast 2019 - Model 2')
plt.legend(loc='best')
plt.xlabel('date')
plt.ylabel('sales')
plt.title('Forecasts using Holt-Winters with damping trend component')
plt.show()

plt.figure(figsize=(11,5))
plt.plot(errors_df_hwd['date'], np.abs(errors_df_hwd['errors']), label='errors')
plt.plot(errors_df_hwd['date'], errors_df_hwd['sales'], label='actual sales')
plt.plot(errors_df_hwd['date'], errors_df_hwd['pred_sales'], label='forecast')
plt.legend(loc='best')
plt.xlabel('date')
plt.ylabel('sales')
plt.title('Holt-Winters (damping) forecasts with actual sales and errors')
plt.show()

result_df_hwd = errors_df_hwd.groupby('model').agg(total_sales=('sales', 'sum'),
                                          total_pred_sales=('pred_sales', 'sum'),
                                          holt_winters_overall_error=('errors', 'sum'),
                                          MAE=('errors', mae),
                                          RMSE=('errors', rmse), 
                                          MAPE=('errors', mape))
result_df_hwd

NameError: name 'pred_fit_2' is not defined

# 3. ARMIA Model: Autoregressive Integrated Moving Average

In [49]:
arima_df = train[['date', 'sales']].set_index('date')
arima_test_df = test[['date', 'sales']].set_index('date')

def test_stationarity(timeseries):
    # Plotting rolling statistics
    rollmean = timeseries.rolling(window=365).mean()
    rollstd = timeseries.rolling(window=365).std()
    adfuller_test = adfuller(timeseries, autolag='AIC')
    print("Test statistic = {:.3f}".format(adfuller_test[0]))
    print("P-value = {:.3f}".format(adfuller_test[1]))
    print("Critical values :")
    
    for key, value in adfuller_test[4].items():
        print("\t{}: {} - The data is {} stationary with {}% confidence"
              .format(key, value, '' if adfuller_test[0] < value else 'not', 100-int(key[:-1])))
test_stationarity(arima_df.sales)

KeyError: "['sales'] not in index"

In [50]:
first_difference = arima_df.sales - arima_df.sales.shift(1)
first_difference = pd.DataFrame(first_difference.dropna(inplace=False))
# Check for stationarity after differencing
test_stationarity(first_difference.sales)

NameError: name 'arima_df' is not defined

In [51]:
print(arima_df.shape)
# Remove any rows with missing values
arima_df = arima_df.dropna()
# Extract the endogenous variable
endog = arima_df['sales']
print(len(endog))
# Fit the ARIMA model
arima_model = ARIMA(endog, order=(6, 1, 1)).fit()
print(arima_model.summary())

NameError: name 'arima_df' is not defined

In [52]:
residuals = arima_model.resid
# Checking for seasonality
fig, ax = plt.subplots(2, figsize=(11,5))
ax[0] = plot_acf(residuals, ax=ax[0], lags=40)
ax[1] = plot_pacf(residuals, ax=ax[1], lags=40)

NameError: name 'arima_model' is not defined

In [53]:

# fit the model
sarima_model = SARIMAX(arima_df.sales, order=(6, 1, 0), seasonal_order=(6, 1, 0, 7), 
                       enforce_invertibility=False, enforce_stationarity=False)
sarima_fit = sarima_model.fit()
arima_test_df['pred_sales'] = sarima_fit.predict(start=arima_test_df.index[0],
                                                 end=arima_test_df.index[-1], dynamic= True)

NameError: name 'arima_df' is not defined

In [54]:
# Fit the SARIMA model
sarima_model = SARIMAX(arima_df.sales, order=(6, 1, 0), seasonal_order=(6, 1, 0, 7), 
                       enforce_invertibility=False, enforce_stationarity=False)
sarima_fit = sarima_model.fit()

# Generate the forecast for the next two years
forecast = sarima_fit.get_forecast(steps=24)

# Extract the forecasted sales and dates
forecasted_sales = forecast.predicted_mean
forecast_dates = pd.date_range(start=arima_test_df.index[-1] + pd.DateOffset(months=1), 
                              periods=24, freq='MS')

# Create a DataFrame for the forecasted sales
forecast_df = pd.DataFrame({'date': forecast_dates, 'pred_sales': forecasted_sales})
# eval
arima_test_df['errors'] = arima_test_df.sales - arima_test_df.pred_sales
arima_test_df.insert(0, 'model', 'SARIMA')

# Plot the forecasted sales
plt.figure(figsize=(11, 5))
plt.plot(train['date'], train['sales'], label='Train')
plt.plot(test['date'], test['sales'], label='Test')
plt.plot(forecast_df['date'], forecast_df['pred_sales'], label='Forecast - SARIMA')
plt.legend(loc='best')
plt.xlabel('year')
plt.ylabel('sales')
plt.title('Forecasts using Seasonal ARIMA (SARIMA) model')
plt.show()

NameError: name 'arima_df' is not defined

In [55]:
lt.figure(figsize=(11,5))
plt.plot(arima_test_df.index, np.abs(arima_test_df['errors']), label='errors')
plt.plot(arima_test_df.index, arima_test_df['sales'], label='actual sales')
plt.plot(arima_test_df.index, arima_test_df['pred_sales'], label='forecast')
plt.legend(loc='best')
plt.xlabel('date')
plt.ylabel('sales')
plt.title('Seasonal ARIMA (SARIMA) forecasts with actual sales and errors')
plt.show()

result_df_sarima = arima_test_df.groupby('model').agg(total_sales=('sales', 'sum'),
                                          total_pred_sales=('pred_sales', 'sum'),
                                          SARIMA_overall_error=('errors', 'sum'),
                                          MAE=('errors', mae),
                                          RMSE=('errors', rmse), 
                                          MAPE=('errors', mape))
result_df_sarima

NameError: name 'lt' is not defined